# Random Forest Regression - Using Time Series
https://www.kaggle.com/mailology1/bike-sharing-random-forest-regression
https://towardsdatascience.com/improving-random-forest-in-python-part-1-893916666cd

The purpose of this analysis is to create a model that predicts the suicide rate based on disease rate variables. We have data within the period 2015-2017.

In [122]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import seaborn as sns
import numpy as np

pd.set_option('display.max_columns', 200)

Creating a df with the disease rates for each city and their respective suicide rates in each year.

In [123]:
df_2015 = pd.read_csv('../../../TabNet/complete_rates_2015.csv', index_col=[0])
df_2016 = pd.read_csv('../../../TabNet/complete_rates_2016.csv', index_col=[0])
df_2017 = pd.read_csv('../../../TabNet/complete_rates_2017.csv', index_col=[0])
df_2015['ANO'] = 2015
df_2016['ANO'] = 2016
df_2017['ANO'] = 2017
frames = [df_2015, df_2016, df_2017]
df = pd.concat(frames)

In [124]:
df_2015.shape, df_2016.shape, df_2017.shape

((2995, 17), (3012, 17), (3139, 17))

In [125]:
df

MUNCOD  OSTEOPOROSE  INSUFICIENCIA_RENAL  INSUFICIENCIA_CARDIACA  \
0     110001     3.909610            23.457659              105.559465   
1     110034     5.860634            35.163805              105.491414   
2     110002     7.662762            65.133476               96.742368   
3     110045     5.285692            23.785612               50.214071   
4     110003     0.000000            15.735641              157.356412   
...      ...          ...                  ...                     ...   
3134  430781     0.000000             0.000000              400.534045   
3135  431849     0.000000             0.000000                0.000000   
3136  432162     0.000000            83.647010              627.352572   
3137  432255     0.000000            47.596383                0.000000   
3138  510719     0.000000            43.103448                0.000000   

      HIPERTENSAO_ESSENCIAL  ESCLEROSE_MULTIPLA  EPILEPSIA  ENXAQUECA  \
0                449.605129            3.909610  58.644147  15.638439   
1                105.491414            0.000000   5.860634  17.581902   
2                  2.873536            0.000000  28.735357  12.451988   
3                 10.571383            0.000000  21.142767   2.642846   
4                141.620771            0.000000  31.471282   0.000000   
...                     ...                 ...        ...        ...   
3134             160.213618            0.000000   0.000000   0.000000   
3135               0.000000            0.000000   0.000000   0.000000   
3136               0.000000            0.000000   0.000000   0.000000   
3137               0.000000           47.596383  47.596383   0.000000   
3138               0.000000            0.000000   0.000000   0.000000   

             DPOC  DORSOPATIAS  DOENCA_DE_PARKINSON  DIABETES_MELLITUS  \
0      293.220737     3.909610              3.90961         207.209321   
1       64.466975     5.860634              0.00000         246.146633   
2       68.007011    31.608893              0.00000          81.416845   
3       58.142608    23.785612              0.00000          29.071304   
4       15.735641    31.471282              0.00000          62.942565   
...           ...          ...                  ...                ...   
3134  1068.090788     0.000000              0.00000         213.618158   
3135   362.811791     0.000000              0.00000          90.702948   
3136   334.588038     0.000000              0.00000          41.823505   
3137    95.192765     0.000000              0.00000          95.192765   
3138   560.344828     0.000000              0.00000         215.517241   

          CANCER        ASMA       AIDS       RATE   ANO  
0      89.921026  191.570881   0.000000  11.728829  2015  
1      82.048878   23.442536   5.860634   5.860634  2015  
2      45.976571   43.103035  10.536298   4.789226  2015  
3      55.499762   13.214229   7.928537   2.642846  2015  
4      31.471282   15.735641   0.000000  31.471282  2015  
...          ...         ...        ...        ...   ...  
3134   80.106809  293.724967   0.000000  26.702270  2017  
3135  272.108844    0.000000   0.000000  45.351474  2017  
3136  376.411543    0.000000   0.000000  41.823505  2017  
3137  142.789148   47.596383   0.000000  47.596383  2017  
3138  129.310345    0.000000   0.000000  43.103448  2017  

[9146 rows x 17 columns]

Let's filter out the municipalities that don't have one example in every year.

In [126]:
muncods = (df['MUNCOD'].value_counts() == 3)
muncods = muncods[muncods == True].index
df = df[df['MUNCOD'].isin(muncods)].reset_index(drop=True)

In [127]:
df

MUNCOD  OSTEOPOROSE  INSUFICIENCIA_RENAL  INSUFICIENCIA_CARDIACA  \
0     110001     3.909610            23.457659              105.559465   
1     110002     7.662762            65.133476               96.742368   
2     110045     5.285692            23.785612               50.214071   
3     110004    11.464472            59.615252              130.694976   
4     110080    16.559718            41.399296               86.938522   
...      ...          ...                  ...                     ...   
4831  431300     0.000000             0.000000              179.425837   
4832  120033     0.000000            16.750419               50.251256   
4833  130150     0.000000             5.130310               41.042479   
4834  130370     0.000000             8.667389                8.667389   
4835  432162     0.000000            83.647010              627.352572   

      HIPERTENSAO_ESSENCIAL  ESCLEROSE_MULTIPLA  EPILEPSIA  ENXAQUECA  \
0                449.605129             3.90961  58.644147  15.638439   
1                  2.873536             0.00000  28.735357  12.451988   
2                 10.571383             0.00000  21.142767   2.642846   
3                158.209708             0.00000  49.297228   3.439341   
4                  4.139930             0.00000  37.259367   4.139930   
...                     ...                 ...        ...        ...   
4831               0.000000             0.00000  29.904306  29.904306   
4832               0.000000             0.00000  22.333892   0.000000   
4833              76.954648             0.00000  20.521239   0.000000   
4834              26.002167             0.00000   8.667389   0.000000   
4835               0.000000             0.00000   0.000000   0.000000   

            DPOC  DORSOPATIAS  DOENCA_DE_PARKINSON  DIABETES_MELLITUS  \
0     293.220737     3.909610             3.909610         207.209321   
1      68.007011    31.608893             0.000000          81.416845   
2      58.142608    23.785612             0.000000          29.071304   
3      84.837090    43.564992             3.439341         181.138651   
4      24.839578     0.000000             8.279859          66.238874   
...          ...          ...                  ...                ...   
4831  299.043062     0.000000             0.000000          29.904306   
4832   50.251256     5.583473             0.000000          67.001675   
4833   20.521239     0.000000             0.000000          25.651549   
4834    0.000000    17.334778             0.000000          43.336945   
4835  334.588038     0.000000             0.000000          41.823505   

          CANCER        ASMA       AIDS       RATE   ANO  
0      89.921026  191.570881   0.000000  11.728829  2015  
1      45.976571   43.103035  10.536298   4.789226  2015  
2      55.499762   13.214229   7.928537   2.642846  2015  
3     158.209708   33.246968   1.146447   6.878683  2015  
4      37.259367   16.559718  33.119437  20.699648  2015  
...          ...         ...        ...        ...   ...  
4831  119.617225    0.000000   0.000000  29.904306  2017  
4832   22.333892   16.750419   0.000000   5.583473  2017  
4833   10.260620   10.260620   0.000000  10.260620  2017  
4834    4.333694    4.333694   8.667389  26.002167  2017  
4835  376.411543    0.000000   0.000000  41.823505  2017  

[4836 rows x 17 columns]

### Baseline

To evaluate our model's result, let's establish as baseline the Rate for the last year.

In [128]:
df['Baseline'] = df.groupby('MUNCOD')['RATE'].shift(1)

Let's check the Baseline for some examples:

In [129]:
df[df['MUNCOD'].isin([110001, 110002])]

MUNCOD  OSTEOPOROSE  INSUFICIENCIA_RENAL  INSUFICIENCIA_CARDIACA  \
0     110001     3.909610            23.457659              105.559465   
1     110002     7.662762            65.133476               96.742368   
1612  110001    11.761938             7.841292              129.381322   
1613  110002    29.274005            63.269623              114.263051   
3224  110001    11.793844            31.450250              141.526123   
3225  110002    29.810424           100.610182              138.804788   

      HIPERTENSAO_ESSENCIAL  ESCLEROSE_MULTIPLA  EPILEPSIA  ENXAQUECA  \
0                449.605129            3.909610  58.644147  15.638439   
1                  2.873536            0.000000  28.735357  12.451988   
1612             337.175567            0.000000  39.206461   3.920646   
1613               9.443227            0.944323  28.329682   2.832968   
3224             444.234776            0.000000  35.381531   3.931281   
3225              36.331455            0.000000  21.426242   9.315758   

            DPOC  DORSOPATIAS  DOENCA_DE_PARKINSON  DIABETES_MELLITUS  \
0     293.220737     3.909610              3.90961         207.209321   
1      68.007011    31.608893              0.00000          81.416845   
1612  360.699443     7.841292              0.00000         129.381322   
1613   64.213946    16.997809              0.00000         107.652791   
3224  397.059402     7.862562              0.00000         129.732280   
3225   97.815455    13.973636              0.00000         140.667940   

         CANCER        ASMA       AIDS       RATE   ANO   Baseline  
0     89.921026  191.570881   0.000000  11.728829  2015        NaN  
1     45.976571   43.103035  10.536298   4.789226  2015        NaN  
1612  90.174861  223.476829   0.000000   7.841292  2016  11.728829  
1613  63.269623   54.770718  17.942132   5.665936  2016   4.789226  
3224  98.282030  216.220466   3.931281  11.793844  2017   7.841292  
3225  61.484000   63.347152  18.631515   9.315758  2017   5.665936

In [130]:
from sklearn.metrics import mean_squared_log_error
import math
baseline = df['Baseline']
y_test = df[baseline.notna()]['RATE']
y_pred = df[baseline.notna()]['Baseline']
math.sqrt(mean_squared_log_error(y_test, y_pred))

0.603138700311047

We get 60% of error predicting the risk using the previous year.

In [131]:
df = df.drop(['Baseline'], axis=1)

### Prepare train and validation

Let's divide:

* Train dataset: 2015, 2016, 2017
* Validation dataset: 2018

In [132]:
train = df[df['ANO'] <= 2017]
#valid = df[df['ANO'] > 2017]

In [133]:
#train.shape, valid.shape

### Computing features

* Difference Rate

We compute the difference of the rates RATE_(t) - RATE_(t+1), for the target value and the diseases.

This is useful to make the time series stationary, too.

In [134]:
cols = ['OSTEOPOROSE', 'INSUFICIENCIA_RENAL',
        'INSUFICIENCIA_CARDIACA', 'HIPERTENSAO_ESSENCIAL', 'ESCLEROSE_MULTIPLA',
        'EPILEPSIA', 'ENXAQUECA', 'DPOC', 'DORSOPATIAS', 'DOENCA_DE_PARKINSON',
        'DIABETES_MELLITUS', 'CANCER', 'ASMA', 'AIDS', 'RATE']

In [135]:
def target_diff(x):
    return x.diff().shift(-1)

for col in cols:
    train['DIFF_{}'.format(col)] = train.groupby('MUNCOD')[col].apply(target_diff)

Let's verify the result for some municipalities:

In [136]:
train[train['MUNCOD'] < 110005].sort_values(by=['MUNCOD'])
#df.head()

MUNCOD  OSTEOPOROSE  INSUFICIENCIA_RENAL  INSUFICIENCIA_CARDIACA  \
0     110001     3.909610            23.457659              105.559465   
1612  110001    11.761938             7.841292              129.381322   
3224  110001    11.793844            31.450250              141.526123   
1     110002     7.662762            65.133476               96.742368   
1613  110002    29.274005            63.269623              114.263051   
3225  110002    29.810424           100.610182              138.804788   
3     110004    11.464472            59.615252              130.694976   
1615  110004    14.793404            88.760427              122.899052   
3227  110004    16.947812           118.634684              124.283955   

      HIPERTENSAO_ESSENCIAL  ESCLEROSE_MULTIPLA  EPILEPSIA  ENXAQUECA  \
0                449.605129            3.909610  58.644147  15.638439   
1612             337.175567            0.000000  39.206461   3.920646   
3224             444.234776            0.000000  35.381531   3.931281   
1                  2.873536            0.000000  28.735357  12.451988   
1613               9.443227            0.944323  28.329682   2.832968   
3225              36.331455            0.000000  21.426242   9.315758   
3                158.209708            0.000000  49.297228   3.439341   
1615              56.897709            0.000000  93.312243   4.551817   
3227              36.155332            0.000000  68.921102   5.649271   

            DPOC  DORSOPATIAS  DOENCA_DE_PARKINSON  DIABETES_MELLITUS  \
0     293.220737     3.909610             3.909610         207.209321   
1612  360.699443     7.841292             0.000000         129.381322   
3224  397.059402     7.862562             0.000000         129.732280   
1      68.007011    31.608893             0.000000          81.416845   
1613   64.213946    16.997809             0.000000         107.652791   
3225   97.815455    13.973636             0.000000         140.667940   
3      84.837090    43.564992             3.439341         181.138651   
1615   80.794747    30.724763             1.137954         130.864731   
3227   65.531540    13.558250             1.129854         105.076435   

          CANCER        ASMA       AIDS       RATE   ANO  DIFF_OSTEOPOROSE  \
0      89.921026  191.570881   0.000000  11.728829  2015          7.852329   
1612   90.174861  223.476829   0.000000   7.841292  2016          0.031905   
3224   98.282030  216.220466   3.931281  11.793844  2017               NaN   
1      45.976571   43.103035  10.536298   4.789226  2015         21.611243   
1613   63.269623   54.770718  17.942132   5.665936  2016          0.536420   
3225   61.484000   63.347152  18.631515   9.315758  2017               NaN   
3     158.209708   33.246968   1.146447   6.878683  2015          3.328933   
1615  188.900395   27.310900   5.689771  10.241588  2016          2.154408   
3227  150.270600   67.791248   2.259708   5.649271  2017               NaN   

      DIFF_INSUFICIENCIA_RENAL  DIFF_INSUFICIENCIA_CARDIACA  \
0                   -15.616367                    23.821857   
1612                 23.608957                    12.144801   
3224                       NaN                          NaN   
1                    -1.863853                    17.520682   
1613                 37.340559                    24.541738   
3225                       NaN                          NaN   
3                    29.145174                    -7.795924   
1615                 29.874258                     1.384903   
3227                       NaN                          NaN   

      DIFF_HIPERTENSAO_ESSENCIAL  DIFF_ESCLEROSE_MULTIPLA  DIFF_EPILEPSIA  \
0                    -112.429563                -3.909610      -19.437686   
1612                  107.059210                 0.000000       -3.824930   
3224                         NaN                      NaN             NaN   
1                       6.569692                 0.944323       -0.405675   
1613  

* Sliding average

Let's calculate the average between the rates RATE_(t) and RATE_(t+1), for the target value and the diseases.

In [137]:
for col in cols:
    train['AVG_{}_2_YEARS'.format(col)] = train.groupby('MUNCOD')[col].rolling(2).mean().reset_index(level=0, drop=True)
    train['AVG_{}_3_YEARS'.format(col)] = train.groupby('MUNCOD')[col].rolling(3).mean().reset_index(level=0, drop=True)

And again, we verify the result for some municipalities:

In [138]:
train[train['MUNCOD'] < 110005].sort_values(by=['MUNCOD'])
#df.head()

MUNCOD  OSTEOPOROSE  INSUFICIENCIA_RENAL  INSUFICIENCIA_CARDIACA  \
0     110001     3.909610            23.457659              105.559465   
1612  110001    11.761938             7.841292              129.381322   
3224  110001    11.793844            31.450250              141.526123   
1     110002     7.662762            65.133476               96.742368   
1613  110002    29.274005            63.269623              114.263051   
3225  110002    29.810424           100.610182              138.804788   
3     110004    11.464472            59.615252              130.694976   
1615  110004    14.793404            88.760427              122.899052   
3227  110004    16.947812           118.634684              124.283955   

      HIPERTENSAO_ESSENCIAL  ESCLEROSE_MULTIPLA  EPILEPSIA  ENXAQUECA  \
0                449.605129            3.909610  58.644147  15.638439   
1612             337.175567            0.000000  39.206461   3.920646   
3224             444.234776            0.000000  35.381531   3.931281   
1                  2.873536            0.000000  28.735357  12.451988   
1613               9.443227            0.944323  28.329682   2.832968   
3225              36.331455            0.000000  21.426242   9.315758   
3                158.209708            0.000000  49.297228   3.439341   
1615              56.897709            0.000000  93.312243   4.551817   
3227              36.155332            0.000000  68.921102   5.649271   

            DPOC  DORSOPATIAS  DOENCA_DE_PARKINSON  DIABETES_MELLITUS  \
0     293.220737     3.909610             3.909610         207.209321   
1612  360.699443     7.841292             0.000000         129.381322   
3224  397.059402     7.862562             0.000000         129.732280   
1      68.007011    31.608893             0.000000          81.416845   
1613   64.213946    16.997809             0.000000         107.652791   
3225   97.815455    13.973636             0.000000         140.667940   
3      84.837090    43.564992             3.439341         181.138651   
1615   80.794747    30.724763             1.137954         130.864731   
3227   65.531540    13.558250             1.129854         105.076435   

          CANCER        ASMA       AIDS       RATE   ANO  DIFF_OSTEOPOROSE  \
0      89.921026  191.570881   0.000000  11.728829  2015          7.852329   
1612   90.174861  223.476829   0.000000   7.841292  2016          0.031905   
3224   98.282030  216.220466   3.931281  11.793844  2017               NaN   
1      45.976571   43.103035  10.536298   4.789226  2015         21.611243   
1613   63.269623   54.770718  17.942132   5.665936  2016          0.536420   
3225   61.484000   63.347152  18.631515   9.315758  2017               NaN   
3     158.209708   33.246968   1.146447   6.878683  2015          3.328933   
1615  188.900395   27.310900   5.689771  10.241588  2016          2.154408   
3227  150.270600   67.791248   2.259708   5.649271  2017               NaN   

      DIFF_INSUFICIENCIA_RENAL  DIFF_INSUFICIENCIA_CARDIACA  \
0                   -15.616367                    23.821857   
1612                 23.608957                    12.144801   
3224                       NaN                          NaN   
1                    -1.863853                    17.520682   
1613                 37.340559                    24.541738   
3225                       NaN                          NaN   
3                    29.145174                    -7.795924   
1615                 29.874258                     1.384903   
3227                       NaN                          NaN   

      DIFF_HIPERTENSAO_ESSENCIAL  DIFF_ESCLEROSE_MULTIPLA  DIFF_EPILEPSIA  \
0                    -112.429563                -3.909610      -19.437686   
1612                  107.059210                 0.000000       -3.824930   
3224                         NaN                      NaN             NaN   
1                       6.569692                 0.944323       -0.405675   
1613  

* Lag

Series value in an earlier time.

In [143]:
for col in cols:
    train['LAG_1_{}'.format(col)] = train.groupby('MUNCOD')['RATE'].shift(1)
    train['LAG_2_{}'.format(col)] = train.groupby('MUNCOD')['RATE'].shift(2)

In [144]:
train[train['MUNCOD'] < 110005].sort_values(by=['MUNCOD'])
#df.head()

MUNCOD  OSTEOPOROSE  INSUFICIENCIA_RENAL  INSUFICIENCIA_CARDIACA  \
0     110001     3.909610            23.457659              105.559465   
1612  110001    11.761938             7.841292              129.381322   
3224  110001    11.793844            31.450250              141.526123   
1     110002     7.662762            65.133476               96.742368   
1613  110002    29.274005            63.269623              114.263051   
3225  110002    29.810424           100.610182              138.804788   
3     110004    11.464472            59.615252              130.694976   
1615  110004    14.793404            88.760427              122.899052   
3227  110004    16.947812           118.634684              124.283955   

      HIPERTENSAO_ESSENCIAL  ESCLEROSE_MULTIPLA  EPILEPSIA  ENXAQUECA  \
0                449.605129            3.909610  58.644147  15.638439   
1612             337.175567            0.000000  39.206461   3.920646   
3224             444.234776            0.000000  35.381531   3.931281   
1                  2.873536            0.000000  28.735357  12.451988   
1613               9.443227            0.944323  28.329682   2.832968   
3225              36.331455            0.000000  21.426242   9.315758   
3                158.209708            0.000000  49.297228   3.439341   
1615              56.897709            0.000000  93.312243   4.551817   
3227              36.155332            0.000000  68.921102   5.649271   

            DPOC  DORSOPATIAS  DOENCA_DE_PARKINSON  DIABETES_MELLITUS  \
0     293.220737     3.909610             3.909610         207.209321   
1612  360.699443     7.841292             0.000000         129.381322   
3224  397.059402     7.862562             0.000000         129.732280   
1      68.007011    31.608893             0.000000          81.416845   
1613   64.213946    16.997809             0.000000         107.652791   
3225   97.815455    13.973636             0.000000         140.667940   
3      84.837090    43.564992             3.439341         181.138651   
1615   80.794747    30.724763             1.137954         130.864731   
3227   65.531540    13.558250             1.129854         105.076435   

          CANCER        ASMA       AIDS       RATE   ANO  DIFF_OSTEOPOROSE  \
0      89.921026  191.570881   0.000000  11.728829  2015          7.852329   
1612   90.174861  223.476829   0.000000   7.841292  2016          0.031905   
3224   98.282030  216.220466   3.931281  11.793844  2017               NaN   
1      45.976571   43.103035  10.536298   4.789226  2015         21.611243   
1613   63.269623   54.770718  17.942132   5.665936  2016          0.536420   
3225   61.484000   63.347152  18.631515   9.315758  2017               NaN   
3     158.209708   33.246968   1.146447   6.878683  2015          3.328933   
1615  188.900395   27.310900   5.689771  10.241588  2016          2.154408   
3227  150.270600   67.791248   2.259708   5.649271  2017               NaN   

      DIFF_INSUFICIENCIA_RENAL  DIFF_INSUFICIENCIA_CARDIACA  \
0                   -15.616367                    23.821857   
1612                 23.608957                    12.144801   
3224                       NaN                          NaN   
1                    -1.863853                    17.520682   
1613                 37.340559                    24.541738   
3225                       NaN                          NaN   
3                    29.145174                    -7.795924   
1615                 29.874258                     1.384903   
3227                       NaN                          NaN   

      DIFF_HIPERTENSAO_ESSENCIAL  DIFF_ESCLEROSE_MULTIPLA  DIFF_EPILEPSIA  \
0                    -112.429563                -3.909610      -19.437686   
1612                  107.059210                 0.000000       -3.824930   
3224                         NaN                      NaN             NaN   
1                       6.569692                 0.944323       -0.405675   
1613  

## Strategy 1: don't remove anything

Selecting X (independent variables) and y (dependent variable).

In [61]:
X = df.iloc[:,0:14].values
y = df.iloc[:, -1].values

RMSE: 9.767825295936131

Accuracy: 0.43018490996286357

## Strategy 2: imput values

In [23]:
import numpy as np
from sklearn.impute import SimpleImputer
df.loc[:, df.columns != 'RATE'] = df.loc[:, df.columns != 'RATE'].replace(0, np.nan)

X = df.iloc[:, 0:14].values
y = df.iloc[:, -1].values
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X)
X = imputer.transform(X)

{'max_features': 'log2', 'n_estimators': 300}

RMSE: 9.453248697063861

Accuracy: 0.4555252020918406

## Strategy 3: remove rows + fill null

In [35]:
import numpy as np
N = 12
df.loc[:, df.columns != 'RATE'] = df.loc[:, df.columns != 'RATE'].replace(0, np.nan)
df = df.dropna(thresh=N) # At least N non null items
df = df.fillna(0)
X = df.iloc[:, 0:14].values
y = df.iloc[:, -1].values

{'max_features': 'log2', 'n_estimators': 300}

RMSE: 5.910890360138005

Accuracy: 0.20317934464122725

## Strategy 4: remove rows + imput values

In [83]:
import numpy as np
from sklearn.impute import SimpleImputer
N = 12
df.loc[:, df.columns != 'RATE'] = df.loc[:, df.columns != 'RATE'].replace(0, np.nan)
df = df.dropna(thresh=N) # At least N non null items

X = df.iloc[:, 0:14].values
y = df.iloc[:, -1].values

imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X)
X = imputer.transform(X)

{'max_features': 'sqrt', 'n_estimators': 200}

RMSE: 5.923165150865993

Accuracy: 0.19423204398310187

## Continue

Splitting the dataset into the Training set and Test set

In [84]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

Selecting best parameters

In [48]:
# Grid Search
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

regressor = RandomForestRegressor()
parameters = [{'n_estimators' : [150,200,250,300], 'max_features' : ['auto','sqrt','log2']}]
grid_search = GridSearchCV(estimator = regressor, param_grid = parameters)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [49]:
best_parameters

{'max_features': 'sqrt', 'n_estimators': 200}

Random Forest Regression model
Use the best parameters found from above to build the model

In [85]:
regressor = RandomForestRegressor(n_estimators = 200, max_features = 'sqrt') 
regressor.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=200, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

Predicting the values 

In [86]:
y_pred = regressor.predict(X_test) 

Comparing predicted values with true values in testing set

In [77]:
from sklearn.metrics import mean_squared_error
import math
math.sqrt(mean_squared_error(y_test, y_pred))

5.935928279196662

In [78]:
regressor.score(X_test, y_test)

0.24587410626598624

In [79]:
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(estimator = regressor, X = X_train, y = y_train, cv =10)
accuracy.mean()

0.19374755791178383

In [87]:
feature_importance = regressor.feature_importances_
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
plt.figure(figsize=(12,10))
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, df.columns[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

In [69]:
# Get numerical feature importances
importances = list(regressor.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(list(df.columns), importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

Variable: CANCER               Importance: 0.13
Variable: OSTEOPOROSE          Importance: 0.1
Variable: EPILEPSIA            Importance: 0.1
Variable: DPOC                 Importance: 0.1
Variable: INSUFICIENCIA_RENAL  Importance: 0.09
Variable: INSUFICIENCIA_CARDIACA Importance: 0.09
Variable: DIABETES_MELLITUS    Importance: 0.09
Variable: ASMA                 Importance: 0.08
Variable: HIPERTENSAO_ESSENCIAL Importance: 0.07
Variable: AIDS                 Importance: 0.06
Variable: DORSOPATIAS          Importance: 0.05
Variable: ENXAQUECA            Importance: 0.03
Variable: ESCLEROSE_MULTIPLA   Importance: 0.01
Variable: DOENCA_DE_PARKINSON  Importance: 0.01


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [80]:
plt.figure(figsize=(10,6))
plt.scatter(y_test, y_pred)
plt.xlabel('True')
plt.ylabel('Predicted')

Text(0, 0.5, 'Predicted')